In [ ]:
import os
import cv2
import numpy as np
from skimage.io import imread
from skimage.transform import resize
from skimage.feature import hog
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

In [ ]:
import os
from zipfile import ZipFile


!pip install kaggle


os.environ['KAGGLE_USERNAME'] = "cruk12"
os.environ['KAGGLE_KEY'] = "f6ac77148464b981fcc85de69da4347e"


!kaggle datasets download -d jessicali9530/lfw-dataset


with ZipFile('lfw-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('lfw-dataset')


os.remove('lfw-dataset.zip')


 99% 111M/112M [00:06<00:00, 22.5MB/s]
100% 112M/112M [00:06<00:00, 19.2MB/s]


In [ ]:

def extract_features(image_path, model):
    image = Image.open(image_path).convert('RGB')
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = preprocess(image)
    image = image.unsqueeze(0)
    with torch.no_grad():
        features = model(image)
    features = features.squeeze(0)
    return features


def compute_hog(img):
    resized_img = resize(img, (128*4, 64*4))
    fd, hog_image = hog(resized_img, orientations=9, pixels_per_cell=(8, 8),
                    cells_per_block=(2, 2), visualize=True, multichannel=True)
    return fd


def get_pixel(img, center, x, y):
    new_value = 0
    try:
        if img[x][y] >= center:
            new_value = 1
    except:
        pass
    return new_value

def lbp_calculated_pixel(img, x, y):
    center = img[x][y]
    val_ar = []
    val_ar.append(get_pixel(img, center, x-1, y+1))
    val_ar.append(get_pixel(img, center, x, y+1))
    val_ar.append(get_pixel(img, center, x+1, y+1))
    val_ar.append(get_pixel(img, center, x+1, y))
    val_ar.append(get_pixel(img, center, x+1, y-1))
    val_ar.append(get_pixel(img, center, x, y-1))
    val_ar.append(get_pixel(img, center, x-1, y-1))
    val_ar.append(get_pixel(img, center, x-1, y))

    power_val = [1, 2, 4, 8, 16, 32, 64, 128]
    val = 0
    for i in range(len(val_ar)):
        val += val_ar[i] * power_val[i]
    return val

def calcLBP(img):
    height, width, channel = img.shape
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_lbp = np.zeros((height, width,3), np.uint8)
    for i in range(0, height):
        for j in range(0, width):
             img_lbp[i, j] = lbp_calculated_pixel(img_gray, i, j)
    hist_lbp = cv2.calcHist([img_lbp], [0], None, [256], [0, 256])
    return hist_lbp.flatten()


In [ ]:
resnet = models.resnet50(pretrained=True)
resnet = nn.Sequential(*list(resnet.children())[:-1])
resnet.eval()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 137MB/s]


Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [ ]:
lfw_folder = '/content/lfw-dataset/lfw-deepfunneled/lfw-deepfunneled'
X, y = [], []
for folder_name in os.listdir(lfw_folder):
    folder_path = os.path.join(lfw_folder, folder_name)
    if os.path.isdir(folder_path):
        num_images = len(os.listdir(folder_path))
        if num_images > 70:
            for image_name in os.listdir(folder_path):
                image_path = os.path.join(folder_path, image_name)
                image = imread(image_path)
                # hog_feature = compute_hog(image)
                # lbp_feature = calcLBP(image)
                cnn_feature = extract_features(image_path, resnet).numpy()
                # hog_feature = hog_feature.reshape(-1)
                # lbp_feature = lbp_feature.reshape(-1)
                cnn_feature = cnn_feature.flatten()
                # combined_feature = np.concatenate((cnn_feature))

                X.append(cnn_feature)
                y.append(folder_name)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=30)



In [ ]:
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [ ]:
knn_clf = KNeighborsClassifier()
knn_clf.fit(X_train, y_train_encoded)
y_pred_knn = knn_clf.predict(X_test)
accuracy_knn = accuracy_score(y_test_encoded, y_pred_knn)
print("KNN Classifier")
print("Accuracy:", accuracy_knn)
print("Classification Report:")
print(classification_report(y_test_encoded, y_pred_knn, target_names=label_encoder.classes_))


KNN Classifier
Accuracy: 0.6124031007751938
Classification Report:
                   precision    recall  f1-score   support

     Ariel_Sharon       0.57      0.50      0.53        16
     Colin_Powell       0.48      0.63      0.55        41
  Donald_Rumsfeld       0.52      0.46      0.49        26
    George_W_Bush       0.69      0.81      0.74       109
Gerhard_Schroeder       0.50      0.32      0.39        19
      Hugo_Chavez       1.00      0.33      0.50        21
       Tony_Blair       0.55      0.42      0.48        26

         accuracy                           0.61       258
        macro avg       0.62      0.50      0.53       258
     weighted avg       0.63      0.61      0.60       258



In [ ]:
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold

In [ ]:
seed = 42

knn_params = {
    "n_neighbors" : range(1,30,2),
    "weights" : ["uniform", "distance"],
    "metric" : ["euclidean", "manhattan", "minkowski"],
    "leaf_size" : range(1,50,5)
}

cv = RepeatedStratifiedKFold(n_splits = 10 , n_repeats = 3, random_state = seed)
grid_search = GridSearchCV(estimator=knn_clf, param_grid=knn_params, n_jobs = 1, cv = cv, scoring = "accuracy", error_score = 0)
grid_results = grid_search.fit(X_train, y_train_encoded)

final_model = knn_clf.set_params(**grid_results.best_params_)
final_model.fit(X_train, y_train_encoded)
y_pred = final_model.predict(X_test)

accuracy_knn_gd = accuracy_score(y_test_encoded, y_pred)
print("Accuracy:", accuracy_knn_gd)
print(grid_results.best_params_)


Accuracy: 0.6356589147286822
{'leaf_size': 1, 'metric': 'manhattan', 'n_neighbors': 7, 'weights': 'distance'}


In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.95)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)


knn_clf = KNeighborsClassifier()
knn_clf.fit(X_train_pca, y_train_encoded)
y_pred_knn = knn_clf.predict(X_test_pca)
accuracy_knn = accuracy_score(y_test_encoded, y_pred_knn)

print("KNN Classifier")
print("Accuracy:", accuracy_knn)
print("Classification Report:")
print(classification_report(y_test_encoded, y_pred_knn, target_names=label_encoder.classes_))

KNN Classifier
Accuracy: 0.6162790697674418
Classification Report:
                   precision    recall  f1-score   support

     Ariel_Sharon       0.50      0.50      0.50        16
     Colin_Powell       0.49      0.63      0.55        41
  Donald_Rumsfeld       0.57      0.46      0.51        26
    George_W_Bush       0.68      0.82      0.74       109
Gerhard_Schroeder       0.50      0.26      0.34        19
      Hugo_Chavez       0.88      0.33      0.48        21
       Tony_Blair       0.60      0.46      0.52        26

         accuracy                           0.62       258
        macro avg       0.60      0.50      0.52       258
     weighted avg       0.62      0.62      0.60       258



In [ ]:
seed = 42

knn_params = {
    "n_neighbors" : range(1,30,2),
    "weights" : ["uniform", "distance"],
    "metric" : ["euclidean", "manhattan", "minkowski"],
    "leaf_size" : range(1,50,5)
}

cv = RepeatedStratifiedKFold(n_splits = 10 , n_repeats = 3, random_state = seed)
grid_search = GridSearchCV(estimator=knn_clf, param_grid=knn_params, n_jobs = 1, cv = cv, scoring = "accuracy", error_score = 0)
grid_results = grid_search.fit(X_train_pca, y_train_encoded)

final_model = knn_clf.set_params(**grid_results.best_params_)
final_model.fit(X_train_pca, y_train_encoded)
y_pred = final_model.predict(X_test_pca)

accuracy_knn_gd = accuracy_score(y_test_encoded, y_pred)
print("Accuracy:", accuracy_knn_gd)
print(grid_results.best_params_)


Accuracy: 0.6434108527131783
{'leaf_size': 1, 'metric': 'euclidean', 'n_neighbors': 5, 'weights': 'distance'}


In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
X_train_lda = lda.fit_transform(X_train, y_train)
X_test_lda = lda.transform(X_test)

knn_clf = KNeighborsClassifier()
knn_clf.fit(X_train_lda, y_train_encoded)
y_pred_knn = knn_clf.predict(X_test_lda)
accuracy_knn = accuracy_score(y_test_encoded, y_pred_knn)

print("KNN Classifier")
print("Accuracy:", accuracy_knn)
print("Classification Report:")
print(classification_report(y_test_encoded, y_pred_knn, target_names=label_encoder.classes_))

KNN Classifier
Accuracy: 0.7790697674418605
Classification Report:
                   precision    recall  f1-score   support

     Ariel_Sharon       1.00      0.62      0.77        16
     Colin_Powell       0.80      0.80      0.80        41
  Donald_Rumsfeld       0.70      0.81      0.75        26
    George_W_Bush       0.81      0.84      0.83       109
Gerhard_Schroeder       0.59      0.68      0.63        19
      Hugo_Chavez       0.93      0.67      0.78        21
       Tony_Blair       0.69      0.69      0.69        26

         accuracy                           0.78       258
        macro avg       0.79      0.73      0.75       258
     weighted avg       0.79      0.78      0.78       258



In [ ]:
seed = 42

knn_params = {
    "n_neighbors" : range(1,30,2),
    "weights" : ["uniform", "distance"],
    "metric" : ["euclidean", "manhattan", "minkowski"],
    "leaf_size" : range(1,50,5)
}

cv = RepeatedStratifiedKFold(n_splits = 10 , n_repeats = 3, random_state = seed)
grid_search = GridSearchCV(estimator=knn_clf, param_grid=knn_params, n_jobs = 1, cv = cv, scoring = "accuracy", error_score = 0)
grid_results = grid_search.fit(X_train_lda, y_train_encoded)

final_model = knn_clf.set_params(**grid_results.best_params_)
final_model.fit(X_train_lda, y_train_encoded)
y_pred = final_model.predict(X_test_lda)

accuracy_knn_gd = accuracy_score(y_test_encoded, y_pred)
print("Accuracy:", accuracy_knn_gd)
print(grid_results.best_params_)


Accuracy: 0.7945736434108527
{'leaf_size': 1, 'metric': 'euclidean', 'n_neighbors': 3, 'weights': 'uniform'}


In [ ]:
nb_clf = GaussianNB()
nb_clf.fit(X_train, y_train_encoded)
y_pred_nb = nb_clf.predict(X_test)
accuracy_nb = accuracy_score(y_test_encoded, y_pred_nb)
print("Naive Bayes Classifier")
print("Accuracy:", accuracy_nb)
print("Classification Report:")
print(classification_report(y_test_encoded, y_pred_nb, target_names=label_encoder.classes_))


Naive Bayes Classifier
Accuracy: 0.49612403100775193
Classification Report:
                   precision    recall  f1-score   support

     Ariel_Sharon       0.64      0.44      0.52        16
     Colin_Powell       0.54      0.71      0.61        38
  Donald_Rumsfeld       0.24      0.88      0.37        26
    George_W_Bush       0.97      0.28      0.43       109
Gerhard_Schroeder       0.67      0.33      0.44        18
      Hugo_Chavez       0.83      0.48      0.61        21
       Tony_Blair       0.52      0.83      0.64        30

         accuracy                           0.50       258
        macro avg       0.63      0.56      0.52       258
     weighted avg       0.73      0.50      0.50       258



In [ ]:
nb_clf = GaussianNB()
nb_clf.fit(X_train_pca, y_train_encoded)
y_pred_nb = nb_clf.predict(X_test_pca)
accuracy_nb = accuracy_score(y_test_encoded, y_pred_nb)
print("Naive Bayes Classifier")
print("Accuracy:", accuracy_nb)
print("Classification Report:")
print(classification_report(y_test_encoded, y_pred_nb, target_names=label_encoder.classes_))


Naive Bayes Classifier
Accuracy: 0.40310077519379844
Classification Report:
                   precision    recall  f1-score   support

     Ariel_Sharon       0.00      0.00      0.00        16
     Colin_Powell       0.37      0.42      0.40        38
  Donald_Rumsfeld       0.21      0.23      0.22        26
    George_W_Bush       0.44      0.75      0.55       109
Gerhard_Schroeder       0.00      0.00      0.00        18
      Hugo_Chavez       0.00      0.00      0.00        21
       Tony_Blair       0.00      0.00      0.00        30

         accuracy                           0.40       258
        macro avg       0.15      0.20      0.17       258
     weighted avg       0.26      0.40      0.31       258



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
